<a href="https://colab.research.google.com/github/rgrion/mlarrozprevisao/blob/main/API_NASA_Power_South_America.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================
# Clima anual (NASA POWER): T2M (°C) e PRECTOTCORR (mm)
# América do Sul - robusto (annual -> fallback para monthly)
# =============================================

!pip -q install countryinfo requests pandas

import requests, time
import pandas as pd
from countryinfo import CountryInfo
from requests.adapters import HTTPAdapter, Retry

ANOS = list(range(2004, 2024))
PAISES_FAOSTAT = [
    "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
    "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
    "Uruguay", "Venezuela",
]

FALLBACK_COORDS = {
    "Argentina": (-38.4161, -63.6167),
    "Bolivia": (-16.2902, -63.5887),
    "Brazil": (-14.2350, -51.9253),
    "Chile": (-35.6751, -71.5430),
    "Colombia": (4.5709, -74.2973),
    "Ecuador": (-1.8312, -78.1834),
    "Guyana": (4.8604, -58.9302),
    "Paraguay": (-23.4425, -58.4438),
    "Peru": (-9.1900, -75.0152),
    "Suriname": (3.9193, -56.0278),
    "Uruguay": (-32.5228, -55.7658),
    "Venezuela": (6.4238, -66.5897),
}

# Sessão HTTP com retries
session = requests.Session()
retries = Retry(total=3, backoff_factor=1.0, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))
HEADERS = {"User-Agent": "colab-adae-rice/1.0", "Accept": "application/json"}

def get_country_coords_safe(country_name: str):
    try:
        info = CountryInfo(country_name).info()
        latlng = info.get("latlng", None)
        if latlng and len(latlng) == 2:
            return float(latlng[0]), float(latlng[1])
    except Exception:
        pass
    return FALLBACK_COORDS.get(country_name, (None, None))

def nasa_power_annual(lat: float, lon: float, start_year: int, end_year: int):
    url = (
        "https://power.larc.nasa.gov/api/temporal/annual/point"
        f"?parameters=T2M,PRECTOTCORR&community=AG"
        f"&longitude={lon:.4f}&latitude={lat:.4f}"
        f"&start={start_year}&end={end_year}&format=JSON"
    )
    r = session.get(url, headers=HEADERS, timeout=30)
    if r.status_code != 200:
        print(f"[ERRO annual {r.status_code}] ({lat:.2f},{lon:.2f}) => {r.text[:250]}")
        return None
    js = r.json()
    param = js.get("properties", {}).get("parameter", {})
    if "T2M" not in param or "PRECTOTCORR" not in param:
        print(f"[ERRO annual sem parâmetros] ({lat:.2f},{lon:.2f}) keys={list(param.keys())}")
        return None
    df_t = pd.DataFrame(param["T2M"], index=[0]).T.reset_index()
    df_p = pd.DataFrame(param["PRECTOTCORR"], index=[0]).T.reset_index()
    df_t.columns = ["Ano", "Temp_Media_C"]
    df_p.columns = ["Ano", "Precipitacao_mm"]
    df = pd.merge(df_t, df_p, on="Ano")
    df["Ano"] = df["Ano"].astype(int)
    df["Temp_Media_C"] = pd.to_numeric(df["Temp_Media_C"], errors="coerce")
    df["Precipitacao_mm"] = pd.to_numeric(df["Precipitacao_mm"], errors="coerce")
    return df.sort_values("Ano").reset_index(drop=True)

def nasa_power_monthly_then_annual(lat: float, lon: float, start_year: int, end_year: int):
    # Fallback: baixa mensal e agrega por ano (mean de T2M, sum de PRECTOTCORR)
    url = (
        "https://power.larc.nasa.gov/api/temporal/monthly/point"
        f"?parameters=T2M,PRECTOTCORR&community=AG"
        f"&longitude={lon:.4f}&latitude={lat:.4f}"
        f"&start={start_year}&end={end_year}&format=JSON"
    )
    r = session.get(url, headers=HEADERS, timeout=30)
    if r.status_code != 200:
        print(f"[ERRO monthly {r.status_code}] ({lat:.2f},{lon:.2f}) => {r.text[:250]}")
        return None
    js = r.json()
    param = js.get("properties", {}).get("parameter", {})
    if "T2M" not in param or "PRECTOTCORR" not in param:
        print(f"[ERRO monthly sem parâmetros] ({lat:.2f},{lon:.2f}) keys={list(param.keys())}")
        return None

    # param["T2M"] é dict: {'200401': valor, '200402': valor, ...}
    df_t = (
        pd.Series(param["T2M"])
        .rename("Temp_Media_C")
        .to_frame()
        .reset_index()
        .rename(columns={"index": "Ym"})
    )
    df_p = (
        pd.Series(param["PRECTOTCORR"])
        .rename("Precipitacao_mm")
        .to_frame()
        .reset_index()
        .rename(columns={"index": "Ym"})
    )
    df = pd.merge(df_t, df_p, on="Ym")
    # Extrai ano (primeiros 4 dígitos)
    df["Ano"] = df["Ym"].astype(str).str.slice(0, 4).astype(int)
    # Agrega anual: T2M = média; PRECTOTCORR = soma
    anu = df.groupby("Ano").agg(
        Temp_Media_C=("Temp_Media_C", "mean"),
        Precipitacao_mm=("Precipitacao_mm", "sum")
    ).reset_index()
    return anu.sort_values("Ano").reset_index(drop=True)

registros = []
for pais in PAISES_FAOSTAT:
    lat, lon = get_country_coords_safe(pais)
    if lat is None:
        print(f"[PULA] {pais} sem coordenadas.")
        continue

    df = nasa_power_annual(lat, lon, ANOS[0], ANOS[-1])
    if df is None:
        # tenta monthly -> annual
        df = nasa_power_monthly_then_annual(lat, lon, ANOS[0], ANOS[-1])

    if df is None or df.empty:
        print(f"[PULA] Sem dados climáticos para {pais}")
        continue

    df["Pais"] = pais
    registros.append(df[["Pais", "Ano", "Temp_Media_C", "Precipitacao_mm"]])
    time.sleep(0.3)  # pequeno intervalo para evitar throttling

if not registros:
    raise RuntimeError("Não foi possível obter dados climáticos. Verifique os logs acima.")

df_clima = pd.concat(registros, ignore_index=True)
df_clima.to_csv("clima_nasa_power.csv", index=False)
print("OK: clima_nasa_power.csv salvo.")
df_clima.head(12)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.2/602.2 kB 22.5 MB/s eta 0:00:00
[ERRO annual 404] (-34.00,-64.00) => <!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="preload" as="image" href="https://power.larc.nasa.gov/api/website/header/nasa-logo.svg"/><link rel="stylesheet
[ERRO annual 404] (-17.00,-65.00) => <!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="preload" as="image" href="https://power.larc.nasa.gov/api/website/header/nasa-logo.svg"/><link rel="stylesheet
[ERRO annual 404] (-10.00,-55.00) => <!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="preload" as="image" href="https://power.larc.nasa.gov/api/website/header/nasa-logo.svg"/><link rel="stylesheet
[ERRO annual 404] (-30.00,-71.00) => <!DOCTYPE html><ht

,Pais,Ano,Temp_Media_C,Precipitacao_mm
0,Argentina,2004,16.595385,37.50
1,Argentina,2005,16.350769,25.33
2,Argentina,2006,17.235385,31.32
3,Argentina,2007,15.423846,31.62
4,Argentina,2008,16.967692,30.03
5,Argentina,2009,17.474615,30.13
6,Argentina,2010,16.016154,27.39
7,Argentina,2011,16.849231,22.72
8,Argentina,2012,16.526154,42.07
9,Argentina,2013,16.995385,22.20
